#### **Objective:** <br>
Data wrangling of the saved product information. <br>

#### **Method:** <br>
1. Rename columns
2. Apply upper case to string columns 
3. Get the size of the product in ounces
4. Split hierarchy levels into 3 levels/columns
5. Keep only products with hierarchy_1 is skincare
6. Generate dummy variables for skin type and skin concern
7. Save cleaned data into a file
8. Create a file of the products and their similar products/connections

#### **Summary:** <br>
After data preprocessing, there are 1815 skincare products. Each product has a sku_id, hierarchy_1/2/3, brand, product, rating, loves_count, size_oz, price, child_sku, item_id, similar products, skin_concern (dummy variables), skin_type (dummy variables)

In [1]:
#import libaries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import re

##### **1. Data Preprocessing**

In [6]:
#load in the data
raw_skincare_data = pd.read_csv("../results/SephoraSkincareData_v2.csv", sep='\t')

In [7]:
raw_skincare_data.head()

,Unnamed: 0,hierarchy,brand,product,rating,loves_count,reviews_num,size,price,child_sku,item_id,similar_products,skin_concern,skin_type
0,101220,"Skincare,Cleansers,Makeup Removers",Shiseido,Facial Cotton,4.8064,143523,2913.0,165 sheets,16.0,1880350,P173726,"2735132,2031391,2191526,2698348,2031441",NaN,"Normal, Dry, Combination, and Oily"
1,1022318,"Makeup,Lip,Lip Balms & Treatments",GUERLAIN,KissKiss Liplift Lipstick Primer,3.9012,11769,334.0,0.11 oz/ 3 g,40.0,NaN,P190247,"1987478,2595262,2511186,2595940,2608578",NaN,NaN
2,1027465,"Skincare,Treatments,Blemish & Acne Treatments",CLINIQUE,Acne Solutions All-Over Clearing Treatment Oil...,4.0181,18325,719.0,1.7 oz/ 50 mL,26.0,NaN,P188306,"1027473,1677137,1027507,1592831,1592856","Redness, Acne and Blemishes, and Oiliness","Normal, Dry, Combination, and Oily"
3,1027473,"Skincare,Cleansers,Toners",CLINIQUE,Acne Solutions™ Clarifying Lotion,4.3980,46140,1015.0,6.7 oz/ 200 mL,21.0,NaN,P188307,"1027465,1027507,1677137,1592831,1802321","Pores, dullness and uneven texture, acne, and ...","Normal, Dry, Combination, and Oily"
4,1027507,"Skincare,Cleansers,Face Wash & Cleansers",CLINIQUE,Acne Solutions™ Cleansing Foam,4.1654,51136,1052.0,4.2 oz/ 125 mL,25.0,2531747,P188309,"1027473,1027465,1677137,1592831,2531747","Pores, Acne and Blemishes, and Oiliness","Normal, Dry, Combination, and Oily"


In [34]:
clean_skincare_data = raw_skincare_data.copy()

#change "Unnamed:0" col name to item_sku
clean_skincare_data = clean_skincare_data.rename(columns={"Unnamed: 0": "item_sku"})
clean_skincare_data['item_sku'] = clean_skincare_data['item_sku'].astype("string")

#apply upper case to multiple string columns columns
clean_skincare_data['hierarchy'] = clean_skincare_data['hierarchy'].str.upper()
clean_skincare_data['brand'] = clean_skincare_data['brand'].str.upper()
clean_skincare_data['product'] = clean_skincare_data['product'].str.upper()
clean_skincare_data['skin_concern'] = clean_skincare_data['skin_concern'].str.upper()
clean_skincare_data['skin_type'] = clean_skincare_data['skin_type'].str.upper()
clean_skincare_data['size'] = clean_skincare_data['size'].str.upper()

In [35]:
#get the size in oz
size_oz = clean_skincare_data['size'].str.extract(r'(\d+(\.\d+)?)\s*OZ')[0]
clean_skincare_data['size_oz'] = size_oz
clean_skincare_data['size_oz'] = clean_skincare_data['size_oz'].astype(float)

In [36]:
#split the skincare hierachy into 3 levels
clean_skincare_data['hierarchy'] = clean_skincare_data['hierarchy'].str.split(',')

#create a new df where each level is its own column
df_hierarchy = pd.DataFrame([pd.Series(x) for x in clean_skincare_data.hierarchy])
df_hierarchy.columns = ['hierarchy_{}'.format(x+1) for x in df_hierarchy.columns]

#join the new df with the original df
clean_skincare_data = clean_skincare_data.merge(df_hierarchy, left_index=True, right_index=True)



In [37]:
#keep only products that have hierarchy_1 as SKINCARE
clean_skincare_data = clean_skincare_data[clean_skincare_data['hierarchy_1'] == 'SKINCARE']

In [38]:
#exclude products that have hierarchy_2 as "SHOP BY CONCERN" this is only one product
clean_skincare_data = clean_skincare_data[clean_skincare_data['hierarchy_2'] != 'SHOP BY CONCERN']

In [39]:
clean_skincare_data['hierarchy_1'].value_counts()

hierarchy_1
SKINCARE    2135
Name: count, dtype: int64

In [40]:
clean_skincare_data['hierarchy_2'].value_counts()

hierarchy_2
MOISTURIZERS              509
TREATMENTS                411
CLEANSERS                 335
EYE CARE                  169
VALUE & GIFT SETS         168
MASKS                     134
SUNSCREEN                 101
MINI SIZE                  84
HIGH TECH TOOLS            66
LIP BALMS & TREATMENTS     65
WELLNESS                   52
SELF TANNERS               41
Name: count, dtype: int64

In [41]:
clean_skincare_data['hierarchy_3'].value_counts()

hierarchy_3
MOISTURIZERS                 370
FACE SERUMS                  336
FACE WASH & CLEANSERS        207
EYE CREAMS & TREATMENTS      155
FACE MASKS                   107
FACE SUNSCREEN                88
TONERS                        73
FACE OILS                     55
MISTS & ESSENCES              52
FACIAL PEELS                  41
ANTI-AGING                    41
EXFOLIATORS                   37
BLEMISH & ACNE TREATMENTS     32
FOR BODY                      27
SHEET MASKS                   26
BEAUTY SUPPLEMENTS            23
NIGHT CREAMS                  20
FACIAL ROLLERS                18
EYE MASKS                     15
FOR FACE                      14
HOLISTIC WELLNESS             11
BODY SUNSCREEN                10
MAKEUP REMOVERS                9
FACIAL CLEANSING BRUSHES       8
DECOLLETE & NECK CREAMS        7
HAIR REMOVAL                   7
FACE WIPES                     3
BB & CC CREAM                  3
TEETH WHITENING                2
BLOTTING PAPERS                

In [42]:
def clean_concerns(s):
    split_words = [word.strip().replace('.', '').replace('OF', '') 
                    for part in re.split(',| AND|/', s) for word in part.strip().split()]
    return [word for word in split_words if word]

In [43]:
#convert skin_concern into dummy variables
clean_skincare_data['skin_concern'] = clean_skincare_data['skin_concern'].fillna('UNKNOWN')

skin_concerns = [
    'UNKNOWN',
    'ACNE',
    'BLEMISHES',
    'OILINESS',
    'PORES',
    'FINE LINES',
    'WRINKLES',
    'DULLNESS',
    'FIRMNESS',
    'ELASTICITY',
    'UNEVEN TEXTURE',
    'DRYNESS',
    'MASK',
    'DARK SPOTS',
    'PUFFINESS',
    'DARK CIRCLES',
    'BUMPS',
    'INGROWNS',
    'UNEVEN TONE']

for concern in skin_concerns:
    clean_skincare_data[concern] = clean_skincare_data['skin_concern'].str.contains(concern, regex=True).astype(int)


In [44]:
#convert skin_type into dummy variables
mlb = MultiLabelBinarizer()
clean_skincare_data['skin_type'] = clean_skincare_data['skin_type'].fillna('SKIN_TYPE_UNKNOWN')
clean_skincare_data['skin_type'] = clean_skincare_data['skin_type'].str.split(r',| AND').apply(lambda x: [word.strip() for word in x if word.strip()])
df_type = pd.DataFrame(mlb.fit_transform(clean_skincare_data['skin_type']),columns=mlb.classes_)

clean_skincare_data = clean_skincare_data.merge(df_type, left_index=True, right_index=True)

##### **2. Network Graph Data Preparation**
Create a file to store information about connections between skincare products by extracting pairs of products and their similar counterparts


In [45]:
#create a new column combining brand and product name
clean_skincare_data['brand_product_name'] = clean_skincare_data['brand'] + ' ' + clean_skincare_data['product']
brand_product_dict = dict(zip(clean_skincare_data['item_sku'], clean_skincare_data['brand_product_name']))

In [46]:
#store the connected nodes/products in a dictionary which will then later be converted to a dataframe and saved as csv
network_dict = {'from_sku': [], 'to_sku': [], 'from_name': [], 'to_name': [], 'from_brand': [], 'to_brand': []}



for current_sku, similar_sku in zip(clean_skincare_data['item_sku'], clean_skincare_data['similar_products']):
    similar_skus = similar_sku.split(",")
    for sku in similar_skus:
        if sku not in brand_product_dict:
            continue
        network_dict['from_sku'].append(current_sku)
        network_dict['from_name'].append(brand_product_dict[current_sku])
        network_dict['to_sku'].append(sku)
        network_dict['to_name'].append(brand_product_dict[sku])

In [47]:
network_df = pd.DataFrame(network_dict)

# Save DataFrame to CSV
network_df.to_csv('../results/product_network_data.csv', index=False)

print("Data saved to 'network_data.csv'")

Data saved to 'network_data.csv'


##### **3. Save Cleaned Skincare Product Info**

In [48]:
#count the number of connected products each product has
connections = network_df.groupby(['from_sku']).size()
connections = connections.to_frame().reset_index() 
connections.rename(columns={0: 'connections_num'}, inplace=True)


final_clean_skincare_data = clean_skincare_data.join(connections.set_index('from_sku'), on='item_sku', how='left')
#fill null values in connections with 0
final_clean_skincare_data['connections_num'] = final_clean_skincare_data['connections_num'].fillna(0)

final_clean_skincare_data.rename(columns={'UNKNOWN': 'SKIN_CONCERN_UNKNOWN'}, inplace=True)

#remove columns: hierarchy,size, skin_type,skin_concern
final_clean_skincare_data.drop(columns=['hierarchy', 'size', 'skin_type', 'skin_concern'], inplace=True)



# Save DataFrame to CSV
final_clean_skincare_data.to_csv('../results/skincare_data_cleaned.csv', index=False)

print("Data saved to 'skincare_data_cleaned.csv'")


Data saved to 'skincare_data_cleaned.csv'
